In [1]:
# Import the dependencies
import pandas as pd
import numpy as np
import datetime

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# ORM Stuff
from sqlalchemy import create_engine, inspect, text, func
# from sqlalchemy.orm import Session
# from sqlalchemy.ext.automap import automap_base

In [2]:
# Load the CSV file
file_path1 = "Datasets/Starbucks_Store_Locations.csv"
file_path2 = "Datasets/continents2.csv"

df_Starbucks = pd.read_csv(file_path1, encoding='unicode_escape')
df_Countries = pd.read_csv(file_path2)
# Display the first few rows to understand its structure
df_Starbucks.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [4]:
# Re-merging with the correct key (using alpha-2 from Starbucks dataset before replacement)
df_merged = df_Starbucks.merge(df_Countries[['alpha-2', 'name', 'region']], 
                               left_on='Country', right_on='alpha-2', how='left')

# Replace the alpha-2 country code with the full country name
df_merged['Country'] = df_merged['name']

# Drop unnecessary columns from the merge
df_merged.drop(columns=['alpha-2', 'name'], inplace=True)


In [6]:
df_merged.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,region
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,Andorra,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,Europe
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,United Arab Emirates,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42,Asia
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,United Arab Emirates,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39,Asia
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,United Arab Emirates,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,Asia
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,United Arab Emirates,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,Asia


In [8]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25601 entries, 0 to 25600
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           25601 non-null  object 
 1   Store Number    25600 non-null  object 
 2   Store Name      25600 non-null  object 
 3   Ownership Type  25600 non-null  object 
 4   Street Address  25598 non-null  object 
 5   City            25584 non-null  object 
 6   State/Province  25599 non-null  object 
 7   Country         25601 non-null  object 
 8   Postcode        24077 non-null  object 
 9   Phone Number    18739 non-null  object 
 10  Timezone        25599 non-null  object 
 11  Longitude       25598 non-null  float64
 12  Latitude        25598 non-null  float64
 13  region          25601 non-null  object 
dtypes: float64(2), object(12)
memory usage: 2.7+ MB


In [10]:
UNWANTED_COLUMNS = ["Store Name", "Brand", "Ownership Type", "Postcode", "Phone Number", "Timezone", "Street Address"]
df_merged = df_merged.drop(columns=UNWANTED_COLUMNS).dropna()
df_merged.head()

,Store Number,City,State/Province,Country,Longitude,Latitude,region
0,47370-257954,Andorra la Vella,7,Andorra,1.53,42.51,Europe
1,22331-212325,Ajman,AJ,United Arab Emirates,55.47,25.42,Asia
2,47089-256771,Ajman,AJ,United Arab Emirates,55.47,25.39,Asia
3,22126-218024,Abu Dhabi,AZ,United Arab Emirates,54.38,24.48,Asia
4,17127-178586,Abu Dhabi,AZ,United Arab Emirates,54.54,24.51,Asia


In [12]:
df_merged.rename(columns = {'Store Number':'store_number', 'City':'city', 'State/Province':'state_province', 'Country':'country', 'Longitude': 'longitude', 'Latitude':'latitude'}, inplace=True)
df_merged

,store_number,city,state_province,country,longitude,latitude,region
0,47370-257954,Andorra la Vella,7,Andorra,1.53,42.51,Europe
1,22331-212325,Ajman,AJ,United Arab Emirates,55.47,25.42,Asia
2,47089-256771,Ajman,AJ,United Arab Emirates,55.47,25.39,Asia
3,22126-218024,Abu Dhabi,AZ,United Arab Emirates,54.38,24.48,Asia
4,17127-178586,Abu Dhabi,AZ,United Arab Emirates,54.54,24.51,Asia
...,...,...,...,...,...,...,...
25596,21401-212072,Thành Ph? H? Chí Minh,SG,Vietnam,106.70,10.78,Asia
25597,24010-226985,Thành Ph? H? Chí Minh,SG,Vietnam,106.71,10.72,Asia
25598,47608-253804,Johannesburg,GT,South Africa,28.04,-26.15,Africa
25599,47640-253809,Menlyn,GT,South Africa,28.28,-25.79,Africa


In [13]:
merged_df.to_csv("Datasets/Starbucks_Clean.csv", index = False)

NameError: name 'merged_df' is not defined

In [10]:
# Write to Database
engine = create_engine("sqlite:///starbucks_store_locations.sqlite")

In [11]:
# Write to SQL (NOTE I AM USING con=engine) THIS IS WHAT WORKS APPARENTLY WHEN WRITING OUT DATA
merged_df.to_sql(name="starbucks_store_locations", con=engine, index=False, if_exists="append", method="multi")

25583

In [12]:
engine.dispose()